In [209]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import Counter
from sklearn.externals import joblib

file = pd.read_csv("reviews.csv")

reviews = file['reviews'].values.tolist()
foods = file['food'].values.tolist()
services = file['service'].values.tolist()
prices = file['price'].values.tolist()
places = file['place'].values.tolist()

tokenize_reviews = []
for review in reviews:
    word_tokens = word_tokenize(review)
    tokenize_reviews.append(word_tokens)
    
stoplist = set(stopwords.words('english'))
filtered_reviews = []
words = []

def RemovePunctAndStopWords(tokens):
    nonPunct = re.compile('.*[A-Za-z0-9].*')  # must contain a letter or digit
    filtered = [w for w in tokens if nonPunct.match(w)]
    #Remove the stopwords from filtered text
    filtered_words = [word for word in filtered if word.lower() not in stoplist]
    frequent_words=['the','and','of','this','am','etc','also','are','were','was','is']
    filtered_words = [word for word in filtered_words if word.lower() not in frequent_words]
    filtered_words=[word.lower() for word in filtered_words]
    return filtered_words

for review in tokenize_reviews:
    review = RemovePunctAndStopWords(review)
    filtered_reviews.append(review)

def ngram_list(word_list, n):
    all_ngrams = list(ngrams(word_list, n))
    ngram_res = []
    for ngram in all_ngrams:
        ngram_res.append(ngram)
    return ngram_res

trigram_result = []
for i in range(len(filtered_reviews)):
    if (len(filtered_reviews[i]) > 2):
        trigram = ngram_list(filtered_reviews[i], 3)
        trigram_result.append(dict(Counter(trigram)))
        
# output_feature = pd.DataFrame(trigram_result)
# output_feature.to_csv('output.csv', index=False, header=False)

# output_feature.drop([28,27,26,25,24,23,22,21,20,19,18,17,16,15], axis = 1, inplace = True)

# output_feature_categorical = pd.Categorical(output_feature)

trigram_set = set()
for trigram in trigram_result:
    for key in trigram:
        trigram_set.add(key)

output_feature = pd.DataFrame(0, columns = trigram_set, index = [i for i in range(len(filtered_reviews))])
for i, trigram in enumerate(trigram_result):
    for k, v in iter(trigram.items()):
        output_feature.at[i,k] += v

output_feature.to_csv('output_encode.csv', index=False, header=False)

food_df = pd.DataFrame(foods)
service_df = pd.DataFrame(services)
price_df = pd.DataFrame(prices)
place_df = pd.DataFrame(places)

In [199]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

def randomForest_fit(x_train, y_train, scoring = 'accuracy'):
    randomForest = RandomForestClassifier(random_state=123)

    hyperparam = {'min_samples_leaf': [3, 5, 7, 9, 13, 17, 21, 27, 33, 41, 50, 60, 80, 100],
                  'max_features': ['sqrt', 'log2', 0.25, 0.5, 0.75], 
                  'n_estimators': [100, 200, 300, 500, 1000]}
    
    random_randomForest = RandomizedSearchCV(randomForest, param_distributions = hyperparam, cv = 5, 
                                           n_iter = 10, scoring = scoring, n_jobs=-1, random_state = 123)
    random_randomForest.fit(x_train, y_train)
    
    print ("Best Accuracy", random_randomForest.best_score_)
    print ("Best Param", random_randomForest.best_params_)
    return random_randomForest

best_randForest = randomForest_fit(output_feature, food_df)

randForest = RandomForestClassifier(random_state=123, n_jobs = -1,
                                   min_samples_leaf = best_randForest.best_params_.get('min_samples_leaf'),
                                   max_features = best_randForest.best_params_.get('max_features'),
                                   n_estimators = best_randForest.best_params_.get('n_estimators'))
randForest.fit(output_feature, food_df)

/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:4

/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/model_selection

Best Accuracy 0.5868596881959911
Best Param {'n_estimators': 300, 'min_samples_leaf': 3, 'max_features': 0.25}


/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.25, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=False, random_state=123, verbose=0, warm_start=False)

In [201]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

def decTree_fit(x_train, y_train, scoring = 'accuracy'):
    decTree = DecisionTreeClassifier(random_state=123)

    hyperparam = {'min_samples_leaf': [3, 5, 7, 9, 13, 17, 21, 27, 33, 41, 50, 60, 80, 100],
                  'max_features': ['sqrt', 'log2', 0.25, 0.5, 0.75]}

    random_decTree = RandomizedSearchCV(decTree, param_distributions = hyperparam, cv = 5,
                                        n_iter = 15, scoring = scoring, n_jobs=-1, random_state = 123)
    
    random_decTree.fit(x_train, y_train)
    
    print ("Best Accuracy", random_decTree.best_score_)
    print ("Best Param", random_decTree.best_params_)
    
    return random_decTree

best_decTree = decTree_fit(output_feature, food_df)

decTree = DecisionTreeClassifier(min_samples_leaf = best_decTree.best_params_.get('min_samples_leaf'),
                                 max_features = best_decTree.best_params_.get('max_features'), random_state=123)
decTree.fit(output_feature, food_df)

Best Accuracy 0.5868596881959911
Best Param {'min_samples_leaf': 33, 'max_features': 'sqrt'}


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=33, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

In [202]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RandomizedSearchCV

def bagging_fit(x_train, y_train, scoring = 'accuracy'):
    decTree = DecisionTreeClassifier(random_state=123)
    
    bagging = BaggingClassifier(base_estimator = decTree, random_state=123)
    
    hyperparam = {'base_estimator__min_samples_leaf': [3, 5, 7, 9, 13, 17, 21, 27, 33, 41, 50, 60, 80, 100],
                  'n_estimators': [100, 200, 300, 500, 1000]}
    # 'base_estimator__' sebelum 'min_samples_leaf' menandakan hyperparameter yang dicari ada di dalam base estimatornya
    # dalam hal ini berarti decTree
    # (min_samples_leaf ada di dalam decTree)
    
    random_bagging = RandomizedSearchCV(bagging, param_distributions = hyperparam, cv = 5, 
                                           n_iter = 10, scoring = scoring, n_jobs=-1, random_state = 123)
    random_bagging.fit(x_train, y_train)
    
    print ("Best Accuracy", random_bagging.best_score_)
    print ("Best Param", random_bagging.best_params_)
    return random_bagging

best_bagging = bagging_fit(output_feature, food_df)

decTreeBag = DecisionTreeClassifier(min_samples_leaf = best_bagging.best_params_.get('base_estimator__min_samples_leaf'),
                                    random_state=123)
bagging = BaggingClassifier(base_estimator = decTreeBag, 
                            n_estimators = best_bagging.best_params_.get('n_estimators'),
                            random_state=123, n_jobs=-1)
bagging.fit(output_feature, food_df)

/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d arr

/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d arr

Best Accuracy 0.5868596881959911
Best Param {'n_estimators': 500, 'base_estimator__min_samples_leaf': 5}


/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=500, n_jobs=-1, oob_score=False,
         random_state=123, verbose=0, warm_start=False)

In [203]:
from sklearn.ensemble import AdaBoostClassifier

def adaBoost_fit(x_train, y_train, scoring = 'accuracy'):
    decTree = DecisionTreeClassifier(random_state=123)
    
    adaBoost = AdaBoostClassifier(base_estimator = decTree, random_state=123)
    
    hyperparam = {'base_estimator__min_samples_leaf': [3, 5, 7, 9, 13, 17, 21, 27, 33, 41, 50, 60, 80, 100],
                  'learning_rate':[1., .1, .01, .001],
                  'n_estimators': [100, 200, 300]}
    
    random_adaBoost = RandomizedSearchCV(adaBoost, param_distributions = hyperparam, cv = 5, 
                                           n_iter = 1, scoring = scoring, n_jobs=-1, random_state = 123)
    random_adaBoost.fit(x_train, y_train)
    
    print ("Best Accuracy", random_adaBoost.best_score_)
    print ("Best Param", random_adaBoost.best_params_)
    return random_adaBoost

best_adaBoost = adaBoost_fit(output_feature, food_df)

decTreeAdaBoost = DecisionTreeClassifier(min_samples_leaf = best_adaBoost.best_params_.get('base_estimator__min_samples_leaf'),
                                    random_state=123)
adaBoost = AdaBoostClassifier(base_estimator = decTreeAdaBoost, 
                            n_estimators = best_adaBoost.best_params_.get('n_estimators'),
                            learning_rate = best_adaBoost.best_params_.get('learning_rate'),
                            random_state=123)
adaBoost.fit(output_feature, food_df)

/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d arr

Best Accuracy 0.5868596881959911
Best Param {'n_estimators': 200, 'learning_rate': 1.0, 'base_estimator__min_samples_leaf': 41}


/Users/keziasuhendra/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=41, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best'),
          learning_rate=1.0, n_estimators=200, random_state=123)

In [210]:
joblib.dump(decTree,'decision_tree.pkl')
joblib.dump(bagging,'bagging.pkl')
joblib.dump(randForest,'random_forest.pkl')
joblib.dump(adaBoost,'adaptive_boosting.pkl')

['adaptive_boosting.pkl']

In [245]:
classifier = joblib.load('decision_tree.pkl')

def predictData(classifier, datas):
    tokenize_data = []
    filtered_data = []
    for review in datas:
        data_tokens = word_tokenize(review)
        tokenize_data.append(data_tokens)
    
    for review in tokenize_data:
        review = RemovePunctAndStopWords(review)
        filtered_data.append(review)
    
    data_result = []
    for i in range(len(filtered_data)):
        if (len(filtered_data[i]) > 2):
            data = ngram_list(filtered_data[i], 3)
            data_result.append(data)

    feature = []
    for trigram in trigram_set:
        feature.append(1 if trigram in data_result[0] else 0)
        
    output_data = pd.DataFrame(feature)
    output_data.to_csv('output_data.csv', index=False, header=False)
    
    result = classifier.predict(output_data.T)
    print(result)
    print(classifier.predict_proba(output_data.T))
    
    return result

predictData(classifier, ["Besides the poor service the good was delicious."])

[0]
[[0.14922049 0.58685969 0.26391982]]


array([0])